In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torchvision
from PIL import Image
from torchvision import transforms as trn
from torch.autograd import Variable as V
import pandas as pd
device = torch.device('cuda')

In [ ]:
def preprocess(image_fname):
    center_crop = trn.Compose([
        trn.Resize((256, 256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_input = Image.open(image_fname)
    img_output = V(center_crop(img_input).unsqueeze(0))
    if torch.cuda.is_available():
      img_output = img_output.cuda()
    return img_output

In [ ]:
def predict(img_path):
    img=preprocess(img_path)
    outputs = model(img)
    max = torch.softmax(outputs, 1)
    if max[0][0] > 0.5:
      masked = 'not masked'
      label = 0
      prob = max[0][0]
    else:
      masked = 'masked'
      label = 1
      prob = max[0][1]
    return label, prob, masked

In [ ]:
def test():
  data_path = 'gdrive/My Drive/CV-FinalProject/dataset/test_real/'
  test = pd.read_csv((data_path + 'test_real.csv'), index_col=0)
  cor = 0
  err = 0
  for i in range(test.shape[0]):
    filename = test.iloc[i, 0]
    label = test.iloc[i, 1]
    rlt, prob, masked = predict((data_path + filename))
    print('{} predicted as {} with probability {:.3f}'.format(filename, masked, prob))
    if rlt == label:
      cor += 1.0
    else:
      err += 1.0
    AP = cor / (cor + err)
    print('Cumulative accuracy: {:.3f}'.format(AP))

**Load the model and replace the output layer with a 2 class layer. Then load the save weights from training. Set to evaluate mode and send to cuda if available.**

In [ ]:
model = torchvision.models.mobilenet_v2()
model.classifier[-1] = nn.Linear(in_features=1280, out_features=2, bias=True)
model.load_state_dict(torch.load('gdrive/My Drive/CV-FinalProject/step2_training/mobilenet_v2_masknet.pth'))
model.eval()
#send to GPU
if torch.cuda.is_available():
  model.cuda()

In [ ]:
test()

im_00.jpg predicted as masked with probability 0.999
Cumulative accuracy: 1.000
im_01.jpg predicted as masked with probability 0.971
Cumulative accuracy: 1.000
im_02.jpg predicted as masked with probability 0.999
Cumulative accuracy: 1.000
im_03.jpg predicted as masked with probability 1.000
Cumulative accuracy: 1.000
im_04.jpg predicted as masked with probability 1.000
Cumulative accuracy: 1.000
im_05.jpg predicted as not masked with probability 0.916
Cumulative accuracy: 0.833
im_06.jpg predicted as masked with probability 0.990
Cumulative accuracy: 0.857
im_07.jpg predicted as masked with probability 0.999
Cumulative accuracy: 0.875
im_08.jpg predicted as masked with probability 0.803
Cumulative accuracy: 0.889
im_09.jpg predicted as masked with probability 1.000
Cumulative accuracy: 0.900
im_10.jpg predicted as masked with probability 1.000
Cumulative accuracy: 0.909
im_11.jpg predicted as masked with probability 0.999
Cumulative accuracy: 0.917
im_12.jpg predicted as masked with p